# Results:



# Purpose:

2015-04-02 (Thursday)

Create code to make tables with info from `argot2` annotations for all genes in a gene-list.

# Shortcuts:

[Convert-lines-from-in_files-to-BED-type](#Convert-lines-from-in_files-to-BED-type)

[Prepare-and-run-the-Bedtools-window-analysis-commands](#Prepare-and-run-the-Bedtools-window-analysis-commands)

[Set-up-annotation-database:](#Set-up-annotation-database:)

[Set-up-gene/SNP-relationship-data:](#Set-up-gene/SNP-relationship-data:)

[Create-tables-for-our-genes-that-reference-the-SNP-location-they-were-associated-with:](#Create-tables-for-our-genes-that-reference-the-SNP-location-they-were-associated-with:)

[Functional-Annotation-of-the-genes-associated-with-the-top-5-environment-SNPs:](#Functional-Annotation-of-the-genes-associated-with-the-top-5-environment-SNPs:)

## Imports:

In [1]:
# imports
import csv

from collections import defaultdict

import pandas as pd

import munch

from IPython.display import display, HTML

import pybedtools as pbt

import sh

from spartan.utils.annotations.ensembl.gff3 import parse_gff3
from spartan.utils.annotations.ensembl.gff3 import parse_gff3_attributes
from spartan.utils.files import tableFile2namedTuple

from spartan.utils.genome_specific.GfusI1 import GfusI1_0

## File paths:

In [14]:
# File Paths

## Functional annatation (Argot2)
fanno = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/annotations/functional/GfusI1.1_pre/argot2_out/argot_functional_annotations_ts150.h5"

## For setting up the BEDTOOLS phase
btools_out_dir = "/home/gus/remote_mounts/louise/data/projects/ddrad58/SNPs_of_interest/2015_04_03_genes_near_SNPs"
btools_gene_models_gff3 = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/annotations/Glossina-fuscipes-IAEA_BASEFEATURES_GfusI1.1.gff3"




## SNPs of interest CSV [ IN FILES ] ----------------------------------------------------------

#### from folder: 2015-03-17_env_selection
Selected_PopPairwiseMSNB_Environm = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSNB_Environm.csv"
Selected_PopPairwiseMSOT_Environm = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSOT_Environm.csv"
Selected_PopPairwiseMSvsNBOT_Environm = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSvsNBOT_Environm.csv"

#### from folder: 2015-02-26_bayescan_tables

Top10_PopPairwiseOverlap_Infection = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_PopPairwiseOverlap_Infection.csv"
Top10_PopPairwiseOverlap_Environm = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_PopPairwiseOverlap_Environm.csv"
Top05_InfectionOverall = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_InfectionOverall.csv"
Top05_PopPairwiseOverlap_Environm = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_PopPairwiseOverlap_Environm.csv"
Top05_PopPairwiseOverlap_Infection = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_PopPairwiseOverlap_Infection.csv"
Top10_InfectionOverall = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_InfectionOverall.csv"


## SNPs of interest BED [ OUT FILES ] ----------------------------------------------------------

#### from folder: 2015-03-17_env_selection
Selected_PopPairwiseMSNB_Environm_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSNB_Environm.bed"
Selected_PopPairwiseMSOT_Environm_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSOT_Environm.bed"
Selected_PopPairwiseMSvsNBOT_Environm_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSvsNBOT_Environm.bed"

#### from folder: 2015-02-26_bayescan_tables
Top10_PopPairwiseOverlap_Infection_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_PopPairwiseOverlap_Infection.bed"
Top10_PopPairwiseOverlap_Environm_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_PopPairwiseOverlap_Environm.bed"
Top05_InfectionOverall_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_InfectionOverall.bed"
Top05_PopPairwiseOverlap_Environm_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_PopPairwiseOverlap_Environm.bed"
Top05_PopPairwiseOverlap_Infection_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_PopPairwiseOverlap_Infection.bed"
Top10_InfectionOverall_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_InfectionOverall.bed"


## Fasta file for renaming contigs ----------------------------------------------------------
fasta = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/assemblies/GfusI1/Glossina-fuscipes-IAEA_SCAFFOLDS_GfusI1.fa"

## outfiles for windowBed analysis ----------------------------------------------------------
window_bed_dir = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection"

In [3]:
def make_BED_line(in_line, name_map):
    chrom = name_map[in_line[1]]
    chromstart = str(int(in_line[2]) - 1)
    chromend = str(int(in_line[2]))
    
    return chrom, chromstart, chromend

def get_in_lines(path, skip_first_row=True):
    with open(path, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        if skip_first_row:
            reader.next()
        for line in reader:
            yield line
            
def write_line(out_file, line):
    line = "%s\n" % ('\t'.join(line))
    out_file.write(line)

def convert_a_file(in_path, out_path, name_map):
    with open(out_path, 'wb') as out_file:
        
        in_lines = get_in_lines(in_path)

        for line in in_lines:
            bed_data = make_BED_line(line, name_map)
            write_line(out_file,bed_data)


# Convert lines from `in_files` to BED type 

In [5]:
name_map = GfusI1_0.get_name_map_from_fasta_headers(fasta)
name_map['KK352174.1']

'Scaffold391'

### _NOTE: I had to do a bit of manual editing of the input files_

1. clean the scaffold name to JUST the supercontig info
2. delete the \*extra\*  column at the bigining of each data line (I think it was the `R` index)

In [6]:
!head $Selected_PopPairwiseMSNB_Environm

In [7]:
convert_a_file(Selected_PopPairwiseMSNB_Environm, Selected_PopPairwiseMSNB_Environm_bed, name_map)

In [8]:
!head $Selected_PopPairwiseMSNB_Environm_bed

Scaffold8	1482580	1482581
Scaffold85	94212	94213
Scaffold107	378348	378349
Scaffold206	259638	259639
Scaffold427	13827	13828
Scaffold23	1150376	1150377
Scaffold69	308923	308924
Scaffold93	365365	365366
Scaffold93	365391	365392
Scaffold144	334226	334227


## Do conversions for remander of files:

In [9]:
convert_a_file(Selected_PopPairwiseMSOT_Environm,
               Selected_PopPairwiseMSOT_Environm_bed,
               name_map)

convert_a_file(Selected_PopPairwiseMSvsNBOT_Environm,
               Selected_PopPairwiseMSvsNBOT_Environm_bed,
               name_map)

convert_a_file(Top10_PopPairwiseOverlap_Infection,
               Top10_PopPairwiseOverlap_Infection_bed,
               name_map)

convert_a_file(Top10_PopPairwiseOverlap_Environm,
               Top10_PopPairwiseOverlap_Environm_bed,
               name_map)

convert_a_file(Top05_InfectionOverall,
               Top05_InfectionOverall_bed,name_map)

convert_a_file(Top05_PopPairwiseOverlap_Environm,
               Top05_PopPairwiseOverlap_Environm_bed,
               name_map)

convert_a_file(Top05_PopPairwiseOverlap_Infection,
               Top05_PopPairwiseOverlap_Infection_bed,
               name_map)

convert_a_file(Top10_InfectionOverall, 
               Top10_InfectionOverall_bed,
               name_map)

# Prepare and run the `Bedtools window` analysis commands

In [21]:
# load bed files into pybedtools objects
Selected_PopPairwiseMSNB_Environm_pbt = pbt.BedTool(Selected_PopPairwiseMSNB_Environm_bed)
Selected_PopPairwiseMSOT_Environm_pbt = pbt.BedTool(Selected_PopPairwiseMSOT_Environm_bed)
Selected_PopPairwiseMSvsNBOT_Environm_pbt = pbt.BedTool(Selected_PopPairwiseMSvsNBOT_Environm_bed)
Top10_PopPairwiseOverlap_Infection_pbt = pbt.BedTool(Top10_PopPairwiseOverlap_Infection_bed)
Top10_PopPairwiseOverlap_Environm_pbt = pbt.BedTool(Top10_PopPairwiseOverlap_Environm_bed)
Top05_InfectionOverall_pbt = pbt.BedTool(Top05_InfectionOverall_bed)
Top05_PopPairwiseOverlap_Environm_pbt = pbt.BedTool(Top05_PopPairwiseOverlap_Environm_bed)
Top05_PopPairwiseOverlap_Infection_pbt = pbt.BedTool(Top05_PopPairwiseOverlap_Infection_bed)
Top10_InfectionOverall_pbt = pbt.BedTool(Top10_InfectionOverall_bed)

# load gene models into pybedtools object and filter for only gene features
btools_gene_models_pbt = pbt.BedTool(btools_gene_models_gff3)
btools_gene_models_pbt_genes = btools_gene_models_pbt.filter(lambda x: x[2] == 'gene').saveas()

In [22]:
# run the windowBed calls

# window == 500
Selected_PopPairwiseMSNB_Environm_win500 = Selected_PopPairwiseMSNB_Environm_pbt.window(btools_gene_models_pbt_genes, w=500)
Selected_PopPairwiseMSOT_Environm_win500 = Selected_PopPairwiseMSOT_Environm_pbt.window(btools_gene_models_pbt_genes, w=500)
Selected_PopPairwiseMSvsNBOT_Environm_win500 = Selected_PopPairwiseMSvsNBOT_Environm_pbt.window(btools_gene_models_pbt_genes, w=500)
Top10_PopPairwiseOverlap_Infection_win500 = Top10_PopPairwiseOverlap_Infection_pbt.window(btools_gene_models_pbt_genes, w=500)
Top10_PopPairwiseOverlap_Environm_win500 = Top10_PopPairwiseOverlap_Environm_pbt.window(btools_gene_models_pbt_genes, w=500)
Top05_InfectionOverall_win500 = Top05_InfectionOverall_pbt.window(btools_gene_models_pbt_genes, w=500)
Top05_PopPairwiseOverlap_Environm_win500 = Top05_PopPairwiseOverlap_Environm_pbt.window(btools_gene_models_pbt_genes, w=500)
Top05_PopPairwiseOverlap_Infection_win500 = Top05_PopPairwiseOverlap_Infection_pbt.window(btools_gene_models_pbt_genes, w=500)
Top10_InfectionOverall_win500 = Top10_InfectionOverall_pbt.window(btools_gene_models_pbt_genes, w=500)

# window == 1000
Selected_PopPairwiseMSNB_Environm_win1000 = Selected_PopPairwiseMSNB_Environm_pbt.window(btools_gene_models_pbt_genes, w=1000)
Selected_PopPairwiseMSOT_Environm_win1000 = Selected_PopPairwiseMSOT_Environm_pbt.window(btools_gene_models_pbt_genes, w=1000)
Selected_PopPairwiseMSvsNBOT_Environm_win1000 = Selected_PopPairwiseMSvsNBOT_Environm_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top10_PopPairwiseOverlap_Infection_win1000 = Top10_PopPairwiseOverlap_Infection_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top10_PopPairwiseOverlap_Environm_win1000 = Top10_PopPairwiseOverlap_Environm_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top05_InfectionOverall_win1000 = Top05_InfectionOverall_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top05_PopPairwiseOverlap_Environm_win1000 = Top05_PopPairwiseOverlap_Environm_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top05_PopPairwiseOverlap_Infection_win1000 = Top05_PopPairwiseOverlap_Infection_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top10_InfectionOverall_win1000 = Top10_InfectionOverall_pbt.window(btools_gene_models_pbt_genes, w=1000)

In [23]:
# save the output

Selected_PopPairwiseMSNB_Environm_win500 = Selected_PopPairwiseMSNB_Environm_win500.moveto("{win_dir}/Selected_PopPairwiseMSNB_Environm_win500.bed".format(win_dir=window_bed_dir))
Selected_PopPairwiseMSOT_Environm_win500 = Selected_PopPairwiseMSOT_Environm_win500.moveto("{win_dir}/Selected_PopPairwiseMSOT_Environm_win500.bed".format(win_dir=window_bed_dir))
Selected_PopPairwiseMSvsNBOT_Environm_win500 = Selected_PopPairwiseMSvsNBOT_Environm_win500.moveto("{win_dir}/Selected_PopPairwiseMSvsNBOT_Environm_win500.bed".format(win_dir=window_bed_dir))
Top10_PopPairwiseOverlap_Infection_win500 = Top10_PopPairwiseOverlap_Infection_win500.moveto("{win_dir}/Top10_PopPairwiseOverlap_Infection_win500.bed".format(win_dir=window_bed_dir))
Top10_PopPairwiseOverlap_Environm_win500 = Top10_PopPairwiseOverlap_Environm_win500.moveto("{win_dir}/Top10_PopPairwiseOverlap_Environm_win500.bed".format(win_dir=window_bed_dir))
Top05_InfectionOverall_win500 = Top05_InfectionOverall_win500.moveto("{win_dir}/Top05_InfectionOverall_win500.bed".format(win_dir=window_bed_dir))
Top05_PopPairwiseOverlap_Environm_win500 = Top05_PopPairwiseOverlap_Environm_win500.moveto("{win_dir}/Top05_PopPairwiseOverlap_Environm_win500.bed".format(win_dir=window_bed_dir))
Top05_PopPairwiseOverlap_Infection_win500 = Top05_PopPairwiseOverlap_Infection_win500.moveto("{win_dir}/Top05_PopPairwiseOverlap_Infection_win500.bed".format(win_dir=window_bed_dir))
Top10_InfectionOverall_win500 = Top10_InfectionOverall_win500.moveto("{win_dir}/Top10_InfectionOverall_win500.bed".format(win_dir=window_bed_dir))
Selected_PopPairwiseMSNB_Environm_win1000 = Selected_PopPairwiseMSNB_Environm_win1000.moveto("{win_dir}/Selected_PopPairwiseMSNB_Environm_win1000.bed".format(win_dir=window_bed_dir))
Selected_PopPairwiseMSOT_Environm_win1000 = Selected_PopPairwiseMSOT_Environm_win1000.moveto("{win_dir}/Selected_PopPairwiseMSOT_Environm_win1000.bed".format(win_dir=window_bed_dir))
Selected_PopPairwiseMSvsNBOT_Environm_win1000 = Selected_PopPairwiseMSvsNBOT_Environm_win1000.moveto("{win_dir}/Selected_PopPairwiseMSvsNBOT_Environm_win1000.bed".format(win_dir=window_bed_dir))
Top10_PopPairwiseOverlap_Infection_win1000 = Top10_PopPairwiseOverlap_Infection_win1000.moveto("{win_dir}/Top10_PopPairwiseOverlap_Infection_win1000.bed".format(win_dir=window_bed_dir))
Top10_PopPairwiseOverlap_Environm_win1000 = Top10_PopPairwiseOverlap_Environm_win1000.moveto("{win_dir}/Top10_PopPairwiseOverlap_Environm_win1000.bed".format(win_dir=window_bed_dir))
Top05_InfectionOverall_win1000 = Top05_InfectionOverall_win1000.moveto("{win_dir}/Top05_InfectionOverall_win1000.bed".format(win_dir=window_bed_dir))
Top05_PopPairwiseOverlap_Environm_win1000 = Top05_PopPairwiseOverlap_Environm_win1000.moveto("{win_dir}/Top05_PopPairwiseOverlap_Environm_win1000.bed".format(win_dir=window_bed_dir))
Top05_PopPairwiseOverlap_Infection_win1000 = Top05_PopPairwiseOverlap_Infection_win1000.moveto("{win_dir}/Top05_PopPairwiseOverlap_Infection_win1000.bed".format(win_dir=window_bed_dir))
Top10_InfectionOverall_win1000 = Top10_InfectionOverall_win1000.moveto("{win_dir}/Top10_InfectionOverall_win1000.bed".format(win_dir=window_bed_dir))

# Set up main data variables:

### Set up annotation database:

In [15]:
argot2 = pd.read_hdf(path_or_buf=fanno, key='dataframe')

In [16]:
argot2.head()

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
0,GFUI034947-PA,P,GO:0006508,proteolysis,270.313447,0.496543,8.247696
1,GFUI035874-PA,F,GO:0005515,protein binding,529.038456,0.500000,5.471582
2,GFUI033625-PA,P,GO:0005980,glycogen catabolic process,307.758251,0.284597,13.424450
3,GFUI033625-PA,F,GO:0004134,4-alpha-glucanotransferase activity,159.513252,0.177063,10.153643
4,GFUI033625-PA,F,GO:0004135,"amylo-alpha-1,6-glucosidase activity",184.352303,0.177063,11.734746


### Set up gene/SNP relationship data:

#### Function to create dictionary-based retrieval object for gene/SNP data:

In [17]:
def bed3_v_gff3_window_by_gff3_ID(bedtools_window_out):
    """
    Returns dictionary-based retrieval object for Bedtools result from 3-field BED format records vs
    GFF3 format records data grouped by the GFF3 features.
    """
    headers = ["bed3_seq",
               "bed3_start",
               "bed3_end",
               "gff3_seq",
               "gff3_source",
               "gff3_type",
               "gff3_start",
               "gff3_end",
               "gff3_score",
               "gff3_strand",
               "gff3_phase",
               "gff3_attributes",]
    
    table = tableFile2namedTuple(bedtools_window_out, 
                                 sep='\t', 
                                 headers=headers)
    
    Tree = lambda: defaultdict(Tree)
    data = Tree()
    
    for row in table:
        gff3_rec = parse_gff3_attributes(row.gff3_attributes)['ID']

        data[gff3_rec]['info'].setdefault('seq', row.gff3_seq)
        data[gff3_rec]['info'].setdefault('source', row.gff3_source)
        data[gff3_rec]['info'].setdefault('type', row.gff3_type)
        data[gff3_rec]['info'].setdefault('start', row.gff3_start)
        data[gff3_rec]['info'].setdefault('end', row.gff3_end)
        data[gff3_rec]['info'].setdefault('score', row.gff3_score)
        data[gff3_rec]['info'].setdefault('strand', row.gff3_strand)
        data[gff3_rec]['info'].setdefault('phase', row.gff3_phase)
        data[gff3_rec]['info'].setdefault('attributes', row.gff3_attributes)

        bed3_hit = dict(seq = row.bed3_seq, 
                        start = row.bed3_start,
                        end = row.bed3_end)

        try:
            data[gff3_rec]['bed3_hits'].append(bed3_hit)

        except AttributeError:
            data[gff3_rec]['bed3_hits'] = []
            data[gff3_rec]['bed3_hits'].append(bed3_hit)

    return munch.munchify(data)

In [35]:
Selected_PopPairwiseMSNB_Environm_win500_search = bed3_v_gff3_window_by_gff3_ID(Selected_PopPairwiseMSNB_Environm_win500.fn)
Selected_PopPairwiseMSOT_Environm_win500_search = bed3_v_gff3_window_by_gff3_ID(Selected_PopPairwiseMSOT_Environm_win500.fn)
Selected_PopPairwiseMSvsNBOT_Environm_win500_search = bed3_v_gff3_window_by_gff3_ID(Selected_PopPairwiseMSvsNBOT_Environm_win500.fn)
Top10_PopPairwiseOverlap_Infection_win500_search = bed3_v_gff3_window_by_gff3_ID(Top10_PopPairwiseOverlap_Infection_win500.fn)
Top10_PopPairwiseOverlap_Environm_win500_search = bed3_v_gff3_window_by_gff3_ID(Top10_PopPairwiseOverlap_Environm_win500.fn)
Top05_InfectionOverall_win500_search = bed3_v_gff3_window_by_gff3_ID(Top05_InfectionOverall_win500.fn)
Top05_PopPairwiseOverlap_Environm_win500_search = bed3_v_gff3_window_by_gff3_ID(Top05_PopPairwiseOverlap_Environm_win500.fn)
Top05_PopPairwiseOverlap_Infection_win500_search = bed3_v_gff3_window_by_gff3_ID(Top05_PopPairwiseOverlap_Infection_win500.fn)
Top10_InfectionOverall_win500_search = bed3_v_gff3_window_by_gff3_ID(Top10_InfectionOverall_win500.fn)
Selected_PopPairwiseMSNB_Environm_win1000_search = bed3_v_gff3_window_by_gff3_ID(Selected_PopPairwiseMSNB_Environm_win1000.fn)
Selected_PopPairwiseMSOT_Environm_win1000_search = bed3_v_gff3_window_by_gff3_ID(Selected_PopPairwiseMSOT_Environm_win1000.fn)
Selected_PopPairwiseMSvsNBOT_Environm_win1000_search = bed3_v_gff3_window_by_gff3_ID(Selected_PopPairwiseMSvsNBOT_Environm_win1000.fn)
Top10_PopPairwiseOverlap_Infection_win1000_search = bed3_v_gff3_window_by_gff3_ID(Top10_PopPairwiseOverlap_Infection_win1000.fn)
Top10_PopPairwiseOverlap_Environm_win1000_search = bed3_v_gff3_window_by_gff3_ID(Top10_PopPairwiseOverlap_Environm_win1000.fn)
Top05_InfectionOverall_win1000_search = bed3_v_gff3_window_by_gff3_ID(Top05_InfectionOverall_win1000.fn)
Top05_PopPairwiseOverlap_Environm_win1000_search = bed3_v_gff3_window_by_gff3_ID(Top05_PopPairwiseOverlap_Environm_win1000.fn)
Top05_PopPairwiseOverlap_Infection_win1000_search = bed3_v_gff3_window_by_gff3_ID(Top05_PopPairwiseOverlap_Infection_win1000.fn)
Top10_InfectionOverall_win1000_search = bed3_v_gff3_window_by_gff3_ID(Top10_InfectionOverall_win1000.fn)

In [36]:
Selected_PopPairwiseMSNB_Environm_win500_search.keys()

['GFUI014656',
 'GFUI014653',
 'GFUI049734',
 'GFUI012007',
 'GFUI049729',
 'GFUI007893',
 'GFUI041284',
 'GFUI015453',
 'GFUI047255',
 'GFUI035185',
 'GFUI022418',
 'GFUI038737',
 'GFUI009284']

In [38]:
len(Selected_PopPairwiseMSNB_Environm_win500_search)

13

In [39]:
Selected_PopPairwiseMSNB_Environm_win500_search.keys()[:3]

['GFUI014656', 'GFUI014653', 'GFUI049734']

In [40]:
Selected_PopPairwiseMSNB_Environm_win500_search['GFUI014656']['bed3_hits']

[Munch(end='869308', seq='Scaffold1', start='869307')]

In [42]:
Selected_PopPairwiseMSNB_Environm_win500_search['GFUI014656'].bed3_hits

[Munch(end='869308', seq='Scaffold1', start='869307')]

## Create tables for our genes that reference the SNP location they were associated with:

In [11]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for index, row in enumerate(self):
            html.append("<tr>")
            
            for col in row:
                # if header, format as such
                if index == 0:
                    html.append("<th>{0}</th>".format(col))
                else:
                    html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

In [14]:
def filter_by_gene(gene, argot_df):
    return argot_df[argot_df.Sequence.str.startswith(gene)]

In [15]:
def sort_by_TS(argot_df):
    return argot_df.sort('Total Score', ascending=0)

In [16]:
def filter_by_TS(tscore, argot_df):
    return argot_df[argot_df["Total Score"] >= tscore]

In [17]:
def gene_table(gene_name, argot_df, tscore):
    return sort_by_TS(filter_by_TS(tscore, filter_by_gene(gene_name, argot_df)))

In [18]:
def vb_link(gene_name, link_template):
    return HTML(link_template % dict(gene_name=gene_name))

In [19]:
def format_snps(snp_list):
    snp_line = "%(seq)s:%(end)s"
    
    snp_strs = []
    
    for snp in snp_list:
        snp_strs.append(snp_line % snp)
    
    br = "<br>"
    return br.join(snp_strs) + br

In [20]:
# Vectorbase Link Templates
gene_page          = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Gene/Summary?db=core;g=%(gene_name)s"> %(gene_name)s: gene home page</a>'''
protein_summary    = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Transcript/ProteinSummary?db=core;g=%(gene_name)s"> %(gene_name)s: protein summary </a>'''
gene_region_detail = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Location/View?db=core;g=%(gene_name)s;r"> %(gene_name)s: genome browser </a>'''
gene_ontology_bp   = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Transcript/Ontology/biological_process?db=core;g=%(gene_name)s;oid=biological_process"> %(gene_name)s: GO: biological process </a>'''
gene_ontology_cp   = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Transcript/Ontology/cellular_component?db=core;g=%(gene_name)s;oid=cellular_component"> %(gene_name)s: GO: cellular component </a>'''
gene_ontology_mf   = '''<a href="https://www.vectorbase.org/Glossina_fuscipes/Transcript/Ontology/molecular_function?db=core;g=%(gene_name)s;oid=molecular_function"> %(gene_name)s: GO: molecular function </a>'''

In [21]:
def present_gene(gene_name, nearby_snps):
    display(HTML("<h2>%s</h2>" % (gene_name)))
    
    nearby_snps = format_snps(nearby_snps)
    display(HTML("<b>SNP location:</b><br>%s" % (nearby_snps)))
    
    display(gene_table(gene_name=gene_name,
                       argot_df=argot2,
                       tscore=200))
    
    display(HTML("<p>"))
    display(vb_link(gene_name=gene_name, link_template=gene_page))
    display(vb_link(gene_name=gene_name, link_template=protein_summary))
    display(vb_link(gene_name=gene_name, link_template=gene_region_detail))
    display(vb_link(gene_name=gene_name, link_template=gene_ontology_bp))
    display(vb_link(gene_name=gene_name, link_template=gene_ontology_cp))
    display(vb_link(gene_name=gene_name, link_template=gene_ontology_mf))
    display(HTML("<hr>"))

# Functional Annotation of the genes associated with the top 5 environment SNPs:

In [22]:
for gene in top5_environment:
    
    snps = top5_environment[gene].bed3_hits
    present_gene(gene_name=gene, nearby_snps=snps)
    

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


...

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
27734,GFUI027255-PA,F,GO:0008199,ferric iron binding,3520.667148,0.294476,8.659780
27735,GFUI027255-PA,C,GO:0005739,mitochondrion,1119.291606,0.695381,5.085303


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
16830,GFUI041872-PA,F,GO:0004896,cytokine receptor activity,259.772571,0.563888,10.488532


# Functional Annotation of the genes associated with the top 5 infection SNPs:

In [23]:
for gene in top5_infection:
    
    snps = top5_infection[gene].bed3_hits
    present_gene(gene_name=gene, nearby_snps=snps)

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
62449,GFUI026694-PA,P,GO:0019307,mannose biosynthetic process,1986.500391,0.185032,15.678973
62453,GFUI026694-PA,C,GO:0005737,cytoplasm,1876.313784,0.773891,3.178476
62451,GFUI026694-PA,F,GO:0004615,phosphomannomutase activity,1509.276050,0.287575,11.148084
62452,GFUI026694-PA,F,GO:0016853,isomerase activity,511.394510,0.545867,4.815628
62450,GFUI026694-PA,F,GO:0003824,catalytic activity,272.359017,0.931151,1.410349


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
65443,GFUI026692-PA,P,GO:0019307,mannose biosynthetic process,2775.627555,0.202663,15.678973
65447,GFUI026692-PA,C,GO:0005737,cytoplasm,2039.739454,0.704821,3.178476
65445,GFUI026692-PA,F,GO:0004615,phosphomannomutase activity,1879.359140,0.298882,11.148084
65446,GFUI026692-PA,F,GO:0016853,isomerase activity,608.639688,0.561327,4.815628
65444,GFUI026692-PA,F,GO:0003824,catalytic activity,397.605948,0.937166,1.410349


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
20316,GFUI051764-PA,F,GO:0003998,acylphosphatase activity,4648.556871,0.426202,10.560419
20317,GFUI051764-PA,F,GO:0016787,hydrolase activity,764.849275,0.753257,2.643033
20319,GFUI051764-PA,C,GO:0070062,extracellular vesicular exosome,537.389311,0.331108,8.572522
20315,GFUI051764-PA,P,GO:0008152,metabolic process,363.072060,0.908530,0.544703
20318,GFUI051764-PA,C,GO:0005739,mitochondrion,260.556240,0.404137,5.085303


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


...

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
44144,GFUI019299-PA,C,GO:0005886,plasma membrane,815.255585,0.151455,4.357169
44146,GFUI019299-PA,C,GO:0016021,integral component of membrane,683.347823,0.153830,3.692502
44145,GFUI019299-PA,C,GO:0016020,membrane,513.407959,0.469204,2.472824
44143,GFUI019299-PA,F,GO:0016740,transferase activity,450.165308,0.498868,2.487400
44141,GFUI019299-PA,F,GO:0004713,protein tyrosine kinase activity,350.882879,0.156891,9.420513
44142,GFUI019299-PA,F,GO:0004714,transmembrane receptor protein tyrosine kinase...,280.651480,0.079869,10.630277
44139,GFUI019299-PA,P,GO:0018108,peptidyl-tyrosine phosphorylation,230.216031,0.030234,12.090538


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content


,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content
